In [6]:
import json
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math

In [7]:
od_data = np.load('od_matrix.npy')
duration_data = np.load('duration_matrix.npy')
distance_data = np.load('distance_matrix.npy')
od_data = od_data[:-3, :-3, :]
duration_data = duration_data[:-3, :-3, :]
distance_data = distance_data[:-3, :-3, :]
delta_c = 1. # energy step [kWh] 0.75
time_granularity = 0.25 # in h
profit_per_mile = 0.8 # in $
cost_per_mile = 1 # in $
charger_capacity = 22.5 # assuming 22.5 kW Chargers
nissan_leave_capacity = 62 # in kWh
charge_time_per_delta_c = math.ceil((delta_c/charger_capacity)/time_granularity)
print(np.sum(od_data))
spring_kwh_per_mi = 11.9 / 100 * 1.609344
energy_distance = np.ceil(((distance_data * spring_kwh_per_mi)/delta_c).max(axis=2))
energy_distance[energy_distance==0] = 1. # we should always use energy to satisfy a trip
print(energy_distance.max())
np.save('energy_distance.npy', energy_distance)
duration_data = np.round(duration_data/(3600*time_granularity)) # convert travel time from sec to min
duration_data[duration_data==0] = 1. # it should always take time to satisfy a trip
data_timespan = od_data.shape[2]
episode_length = int(data_timespan/time_granularity)
fleet_size = 116616 * 2 # got number from Justin Lukes optimization with boundary:283905, without boundary:116616
# number_chargelevels = int(nissan_leave_capacity/delta_c)
number_chargelevels = 10
number_spatial_nodes = 10
print(number_chargelevels)


4427837
2.0
10


In [8]:
new_tripAttr = []
new_reb_time = []
new_total_acc = []
for origin in range(duration_data.shape[0]):
    for destination in range(duration_data.shape[1]):
        for ts in range(episode_length):
            attr = defaultdict()
            attr['time_stamp'] = ts
            attr['origin'] = origin
            attr['destination'] = destination
            attr['demand'] = round(od_data[origin,destination,int(ts*time_granularity)]*time_granularity) # create equal distributed demand over granular time
            attr['price'] = (profit_per_mile + cost_per_mile) * distance_data[origin,destination,int(ts*time_granularity)]
            new_tripAttr.append(attr)

            reb = defaultdict()
            reb['time_stamp'] = ts
            reb['origin'] = origin
            reb['destination'] = destination
            reb['reb_time'] = int(duration_data[origin,destination,int(ts*time_granularity)])
            new_reb_time.append(reb)

for hour in range(24):
    acc = defaultdict()
    acc['hour'] = hour
    acc['acc'] = math.ceil(fleet_size)
    new_total_acc.append(acc)
new_data = defaultdict()
new_data['demand'] = new_tripAttr
new_data['rebTime'] = new_reb_time
new_data['totalAcc'] = new_total_acc
new_data['chargelevels'] = number_chargelevels
new_data['spacialNodes'] = number_spatial_nodes
new_data['chargeTime'] = charge_time_per_delta_c
new_data['episodeLength'] = episode_length

with open('SF_10.json', 'w') as f:
    json.dump(new_data, f)

In [4]:
subproblem_0 = [0,4]
subproblem_1 = [3,7]
subproblem_2 = [6,10]
subproblem_fleet_size = math.ceil(fleet_size/3.)
number_chargelevels_subproblem = 5
number_spatial_nodes_subproblem = subproblem_0[1]-subproblem_0[0]

subproblem_idx = -1
for subproblem in np.array([subproblem_0, subproblem_1, subproblem_2]):
    subproblem_idx += 1
    begin = subproblem[0]
    end = subproblem[1]
    od_subproblem = od_data[begin:end, begin:end, :]
    print(sum(sum(sum(od_subproblem))))
    print(od_subproblem.shape)
    duration_subproblem = duration_data[begin:end, begin:end, :]
    distance_subproblem = distance_data[begin:end, begin:end, :]
    energy_subproblem = energy_distance[begin:end, begin:end]
    new_tripAttr = []
    new_reb_time = []
    new_total_acc = []
    for origin in range(duration_subproblem.shape[0]):
        for destination in range(duration_subproblem.shape[1]):
            for ts in range(episode_length):
                attr = defaultdict()
                attr['time_stamp'] = ts
                attr['origin'] = origin
                attr['destination'] = destination
                attr['demand'] = round(od_subproblem[origin,destination,int(ts*time_granularity)]*time_granularity) # create equal distributed demand over granular time
                attr['price'] = (profit_per_mile + cost_per_mile) * distance_subproblem[origin,destination,int(ts*time_granularity)]
                new_tripAttr.append(attr)

                reb = defaultdict()
                reb['time_stamp'] = ts
                reb['origin'] = origin
                reb['destination'] = destination
                reb['reb_time'] = int(duration_subproblem[origin,destination,int(ts*time_granularity)])
                new_reb_time.append(reb)
    np.save('subproblem_'+str(subproblem_idx)+'_energy_distance.npy', energy_subproblem)
    for hour in range(24):
        acc = defaultdict()
        acc['hour'] = hour
        acc['acc'] = math.ceil(subproblem_fleet_size)
        new_total_acc.append(acc)
    new_data = defaultdict()
    new_data['demand'] = new_tripAttr
    new_data['rebTime'] = new_reb_time
    new_data['totalAcc'] = new_total_acc
    new_data['chargelevels'] = number_chargelevels_subproblem
    new_data['spacialNodes'] = number_spatial_nodes_subproblem
    new_data['chargeTime'] = charge_time_per_delta_c
    new_data['episodeLength'] = episode_length
    print('here',subproblem_fleet_size)

    with open('subproblem_'+str(subproblem_idx)+'_SF_10.json', 'w') as f:
        json.dump(new_data, f)

1362240
(4, 4, 24)
here 38872
1040926
(4, 4, 24)
here 38872
851888
(4, 4, 24)
here 38872
